In [ ]:
import sys
sys.path.append('..')
import h5py
import yaml
import re
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('../flowrec/utils/a4.mplstyle')

from pathlib import Path
from mpl_toolkits.axes_grid1 import ImageGrid

from flowrec.utils import my_discrete_cmap
from flowrec.utils.system import set_gpu
from flowrec.losses import relative_error, momentum_loss, divergence
from flowrec.physics_and_derivatives import vorticity, get_tke
from flowrec.utils.simulation import kolsol_forcing_term
set_gpu(1)
grey = '#808080'

# Define shared functions

In [ ]:
from flowrec.sensors import griddata_periodic
import jax.numpy as jnp

def interpolate(insert_observation_fn, sensor_locs, data_shape, observed):

    sensors_loc_x, sensors_loc_y = sensor_locs

    compare_interp = []
    nt = data_shape[0]
    ndim = data_shape[-1]

    side_length = data_shape[1]
    g1,g2 = np.mgrid[-side_length:side_length*2, -side_length:side_length*2]
    
    temp_observed = np.empty(data_shape)
    temp_observed.fill(np.nan) #this is noisy
    temp_observed = insert_observation_fn(jnp.asarray(temp_observed),jnp.asarray(observed)) # observed_test is noisy if

    for i in range(ndim):
        _locs = np.stack((sensors_loc_x[i].flatten(),sensors_loc_y[i].flatten()),axis=1)
        for t in range(nt):
            _interp = griddata_periodic(_locs,temp_observed[t,...,i][~np.isnan(temp_observed[t,...,i])],(g1,g2),'cubic',side_length)
            compare_interp.append(_interp[side_length:2*side_length,side_length:2*side_length])

    compare_interp = np.array(compare_interp)
    if ndim > 1:
        compare_interp = np.stack((compare_interp[:nt,...],compare_interp[nt:2*nt,...],compare_interp[2*nt:3*nt,...]),axis=-1)

    return compare_interp, temp_observed

In [ ]:
def get_sensor_locs(example_train, take_observation_fn, insert_observation_fn):

    sensors_empty = np.empty_like(example_train[[0],...])
    sensors_empty.fill(np.nan)
    grid_x, grid_y = np.mgrid[0:example_train.shape[1], 0:example_train.shape[2]]

    gridx1 = np.repeat(grid_x[None,:,:,None],3,axis=3)
    gridy1 = np.repeat(grid_y[None,:,:,None],3,axis=3)

    idx_x = take_observation_fn(gridx1)
    idx_y = take_observation_fn(gridy1)

    idx_x = insert_observation_fn(jnp.asarray(sensors_empty),jnp.asarray(idx_x))[0,...]
    sensors_loc_x = []
    for i in range(idx_x.shape[-1]):
        sensors_loc_x.append(idx_x[...,i][~np.isnan(idx_x[...,i])].astype(int))

    idx_y = insert_observation_fn(jnp.asarray(sensors_empty),jnp.asarray(idx_y))[0,...]
    sensors_loc_y = []
    for i in range(idx_y.shape[-1]):
        sensors_loc_y.append(idx_y[...,i][~np.isnan(idx_y[...,i])].astype(int))
    
    return [sensors_loc_x, sensors_loc_y]

In [ ]:
import jax
import flowrec.training_and_states as state_utils
import flowrec.data as data_utils
def get_summary_onecase(d, predict_only=False):

    with open(Path(d,'config.yml'), 'r') as f:
        cfg = yaml.load(f, Loader=yaml.UnsafeLoader)
    cfg.data_config.update({'data_dir':'.'+cfg.data_config.data_dir})
    datacfg = cfg.data_config
    mdlcfg = cfg.model_config
    traincfg = cfg.train_config

    print('Loading data')
    data, datainfo = cfg.case.dataloader(datacfg)

    _keys_to_exclude = [
        'u_train_clean',
        'u_val_clean',
        'train_minmax',
        'val_minmax',
        'u_train',
        'u_val',
        'inn_train',
        'inn_val'
    ]
    observe_kwargs = {key: value for key, value in data.items() if key not in _keys_to_exclude}
    print('Building observation functions')
    take_observation, insert_observation = cfg.case.observe(
        datacfg,
        example_pred_snapshot = data['u_train'][0,...],
        example_pin_snapshot = data['inn_train'][0,...],
        **observe_kwargs
    )
    observed_train, train_minmax = take_observation(data['u_train'], init=True)
    observed_val, val_minmax = take_observation(data['u_val'], init=True)
    data.update({
        'y_train':observed_train, # not normalised
        'y_val':observed_val, # not normalised
        'train_minmax':train_minmax,
        'val_minmax':val_minmax 
    })
    print('Building model')
    prep_data, make_model = cfg.case.select_model(datacfg = datacfg, mdlcfg = mdlcfg, traincfg = traincfg)
    data = prep_data(data, datainfo)
    mdl = make_model(mdlcfg)
    state = state_utils.restore_trainingstate(d,'state')
    inn_train = data['inn_train']
    if datacfg.snr:
        yfull_train_clean = data['u_train_clean']
    else:
        yfull_train_clean = data['u_train']

    print('Predicting')
    pred_train = []
    _t = 0
    while _t < inn_train.shape[0]:
        if (_t + 500) < inn_train.shape[0]:
            pred_train.append(
                mdl.predict(state.params, inn_train[_t:_t+500,:])
            )
        else:
            pred_train.append(
                mdl.predict(state.params, inn_train[_t:,:])
            )
        _t = _t + 500

    pred_train = np.concatenate(pred_train, axis=0)

    if cfg.data_config.normalise:
        pred_train = data_utils.unnormalise_group(pred_train, train_minmax, axis_data=-1, axis_range=0)

    if predict_only:
        return pred_train
    else:
        sensor_locs = get_sensor_locs(yfull_train_clean[:5,...], take_observation, insert_observation)

        print('Interpolating from observations')
        u_interp, observed = interpolate(insert_observation, sensor_locs, yfull_train_clean.shape, observed_train)
    
        # return (clean, noisy, interp, predicted)
        return (yfull_train_clean, data['u_train'], u_interp, pred_train), datainfo, observed

In [ ]:
def make_image_sensor_locations(results, observed, t1, figname):
    fig = plt.figure(figsize=(2,2))
    ax = fig.add_subplot(111)
    ax.imshow(results[0][t1,...,-1].T, alpha=0.3, zorder=1)
    ax.spy(observed[t1,...,-1], color='r', marker='s', markersize=2, alpha=0.6, zorder=2)
    ax.spy(observed[t1,...,0], color='k', marker='s', markersize=2, zorder=5)
    ax.set(xticks=[], xlabel='$x_1$', ylabel='$x_2$', yticks=[])
    if figname:
        fig.savefig('./figs/'+figname)
    else:
        plt.show()

In [ ]:
# def get_tke(ufluc, datainfo):

#     k = np.fft.fftfreq(128,d=datainfo.dx)
#     dk = k[3]-k[2]
#     k = np.fft.fftfreq(128,d=1/128)
#     kgrid1,kgrid2 = np.meshgrid(k,k)
#     kgrid_magnitude = np.sqrt((kgrid1**2)+(kgrid2**2))
#     kgrid_magnitude_int = kgrid_magnitude.astype('int')
#     kmax = np.max(kgrid_magnitude_int)
#     kbins = np.arange(kmax).astype('int')


#     u_fft = np.fft.fft2(ufluc[...,:2],axes=[1,2])
#     ke_fft = np.sum(u_fft * np.conj(u_fft),axis=-1).real * 0.5
#     ke_avg = np.mean(ke_fft,axis=0)
#     spectrum = np.zeros_like(kbins).astype('float32')
#     for i in kbins:
#         spectrum[i] += 0.5*np.sum(ke_avg[kgrid_magnitude_int==i])
    
#     return spectrum, kbins, dk

In [ ]:
p3 = re.compile('^3-')
pmean3 = re.compile('^mean3-')
pclassic = re.compile('^classic-')

def is_loss3(name):
    if re.search(p3, name):
        return True
    else:
        return False
def is_lossmean(name):
    if re.search(pmean3, name):
        return True
    else:
        return False
def is_lossclassic(name):
    if re.search(pclassic, name):
        return True
    else:
        return False

# Clean cases

## Plot summary compare loss functions

In [ ]:
results_dir = Path('../local_results/2dkol/repeat_clean_minimum')


In [ ]:
def read_summary_clean_all(d):

    with h5py.File(Path(d,'summary.h5')) as hf:
        names = np.array(hf.get('runs_name')).astype('unicode')
        l_train = np.array(hf.get('runs_loss_train'))

    loss_total_loss3 = []
    loss_rel_loss3 = []
    name_loss3 = []
    loss_total_mean3 = []
    loss_rel_mean3 = []
    name_mean3 = []
    loss_total_classic = []
    loss_rel_classic = []
    name_classic = []
    for i in range(len(names)):
        if is_loss3(names[i]):
            name_loss3.append(names[i])
            loss_total_loss3.append(np.sum(l_train[i,1:]))
            loss_rel_loss3.append(l_train[i,0])
        if is_lossmean(names[i]):
            name_mean3.append(names[i])
            loss_total_mean3.append(np.sum(l_train[i,1:]))
            loss_rel_mean3.append(l_train[i,0])
        if is_lossclassic(names[i]):
            name_classic.append(names[i])
            loss_total_classic.append(np.sum(l_train[i,1:]))
            loss_rel_classic.append(l_train[i,0])

    loss_total_loss3 = np.array(loss_total_loss3)
    loss_rel_loss3 = np.array(loss_rel_loss3)
    _sort_idx = np.argsort(loss_total_loss3)
    loss_total_loss3 = loss_total_loss3[_sort_idx]
    loss_rel_loss3 = loss_rel_loss3[_sort_idx]
    print('Best run sorted by total loss is ', name_loss3[_sort_idx[0]])
    print('Best run sorted by relative loss is ', name_loss3[np.argsort(loss_rel_loss3)[0]])


    loss_total_mean3 = np.array(loss_total_mean3)
    loss_rel_mean3 = np.array(loss_rel_mean3)
    _sort_idx = np.argsort(loss_total_mean3)
    loss_total_mean3 = loss_total_mean3[_sort_idx]
    loss_rel_mean3 = loss_rel_mean3[_sort_idx]
    print('Best run sorted by total loss is ', name_mean3[_sort_idx[0]])
    print('Best run sorted by relative loss is ', name_mean3[np.argsort(loss_rel_mean3)[0]])

    loss_total_classic = np.array(loss_total_classic)
    loss_rel_classic = np.array(loss_rel_classic)
    _sort_idx = np.argsort(loss_total_classic)
    loss_total_classic = loss_total_classic[_sort_idx]
    loss_rel_classic = loss_rel_classic[_sort_idx]
    print('Best run sorted by total loss is ', name_classic[_sort_idx[0]])
    print('Best run sorted by relative loss is ', name_classic[np.argsort(loss_rel_classic)[0]])

    summary_loss3 = {
        'loss_total': loss_total_loss3,
        'loss_rel': loss_rel_loss3
    }
    summary_mean3 = {
        'loss_total': loss_total_mean3,
        'loss_rel': loss_rel_mean3
    }
    summary_classic = {
        'loss_total': loss_total_classic,
        'loss_rel': loss_rel_classic
    }

    return summary_loss3, summary_mean3, summary_classic

In [ ]:
summary_loss3, summary_mean3, summary_classic = read_summary_clean_all(results_dir)

In [ ]:
clean_example_setup = '224-18'

In [ ]:
total_mean_loss3 = np.mean(summary_loss3['loss_total'])
rel_mean_loss3 = np.mean(summary_loss3['loss_rel'])
rel_std_loss3 = np.std(summary_loss3['loss_rel'])
total_mean_mean3 = np.mean(summary_mean3['loss_total'])
rel_mean_mean3 = np.mean(summary_mean3['loss_rel'])
rel_std_mean3 = np.std(summary_mean3['loss_rel'])
total_mean_classic = np.mean(summary_classic['loss_total'])
rel_mean_classic = np.mean(summary_classic['loss_rel'])
rel_std_classic = np.std(summary_classic['loss_rel'])

print(f'Loss3: {rel_mean_loss3:.4f}+-{rel_std_loss3:.4f}')
print(f'Mean3: {rel_mean_mean3:.4f}+-{rel_std_mean3:.4f}')
print(f'Classic: {rel_mean_classic:.4f}+-{rel_std_classic:.4f}')

In [ ]:
fig, ax = plt.subplots(1,1,sharey=True, figsize=(3,2))

ax.scatter(summary_classic['loss_total'], summary_classic['loss_rel'], label='$\mathcal{L}^c$ training',marker='s', color=my_discrete_cmap(0))
ax.scatter(summary_loss3['loss_total'], summary_loss3['loss_rel'], label='$\mathcal{L}^s$ training',marker='^', color=my_discrete_cmap(1))
ax.scatter(summary_mean3['loss_total'], summary_mean3['loss_rel'], label='$\mathcal{L}^m$ training',marker='.', color=my_discrete_cmap(2))
plt.legend()
ax.set_xlabel('Total loss')
ax.set_ylabel('Rel-l2')

plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
violin_parts = ax.violinplot([summary_classic['loss_rel']*100,summary_loss3['loss_rel']*100,summary_mean3['loss_rel']*100],showmeans=True,showextrema=False)
ax.set_xticks([1,2,3],['$\mathcal{L}^c$','$\mathcal{L}^s$','$\mathcal{L}^m$'])
ax.set_ylabel('$\epsilon$ (\%)')
for i, pc in enumerate(violin_parts['bodies']):
    pc.set_color(my_discrete_cmap(i))
    # pc.set_edgecolor(my_discrete_cmap(i))
# fig.savefig('./figs/2dkol_clean_compare_lossfn')

## Results of cases

In [ ]:
plt_t = 17

## Plot best case

In [ ]:
results_loss3, datainfo, observed = get_summary_onecase(Path(results_dir,'3-'+clean_example_setup))
k_nyquist = (2*np.pi / np.sqrt(2*(datainfo.dx**2))) / 2.

In [ ]:
make_image_sensor_locations(results_loss3, observed, plt_t, '2dkol_clean_sensor_location')

In [ ]:
def make_image_snapshots_clean(data, figname, t1):

    ref = data[0]
    interp = data[2]
    pred = data[3]
    fig = plt.figure(figsize=(6,3))
    grid_1 = ImageGrid(fig, (0.04,0,0.54,0.3), (1,3),cbar_mode='single', share_all=True)
    grid_2 = ImageGrid(fig, (0.04,0.31,0.54,0.3), (1,3),cbar_mode='single', share_all=True)
    grid_3 = ImageGrid(fig, (0.04,0.62,0.54,0.3), (1,3),cbar_mode='single', share_all=True)

    grid_r1 = ImageGrid(fig, (0.64,0,0.36,0.3), (1,2), cbar_mode='single', share_all=True)
    grid_r2 = ImageGrid(fig, (0.64,0.31,0.36,0.3), (1,2), cbar_mode='single', share_all=True)
    grid_r3 = ImageGrid(fig, (0.64,0.62,0.36,0.3), (1,2), cbar_mode='single', share_all=True)

    for i, grid in enumerate([grid_3,grid_2,grid_1]):
        axes = grid.axes_all
        im_ref = axes[0].imshow(ref[t1,...,i].T)
        im_interp = axes[1].imshow(interp[t1,...,i].T)
        im_pred = axes[2].imshow(pred[t1,...,i].T)
        # vmin = []
        # vmax = []
        # for im in [im_ref,im_interp,im_pred]:
        #     clims = im.get_clim()
        #     vmin.append(clims[0])
        #     vmax.append(clims[1])
        for im in [im_ref,im_interp,im_pred]:
            # im.set_clim(min(vmin),max(vmax))
            im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
        grid.cbar_axes[0].colorbar(im_ref)
        grid.axes_all[0].set(xticks=[],yticks=[])

    # errpr
    errinterp = np.abs(ref[t1,...]-interp[t1,...])
    for i, grid in enumerate([grid_r3,grid_r2,grid_r1]):
        imerr_interp = grid.axes_all[0].imshow(errinterp[...,i].T)
        imerr_pred = grid.axes_all[1].imshow(np.abs(ref[t1,...,i]-pred[t1,...,i]).T)
        for im in [imerr_interp, imerr_pred]:
            im.set_clim(imerr_interp.get_clim()[0],imerr_interp.get_clim()[1])
        grid.cbar_axes[0].colorbar(imerr_interp, ticks=[0.0, round(0.5*errinterp[...,i].max(),1)])
        grid.axes_all[0].set(xticks=[],yticks=[])
    fig.text(0,0.15,'$p$')
    fig.text(0,0.48,'$u_2$')
    fig.text(0,0.81,'$u_1$')
    fig.text(0.1,0.97,'Reference')
    fig.text(0.24,0.97,'Interpolated')
    fig.text(0.39,0.97,'Reconstructed')
    fig.text(0.75,0.99,'Absolute error')
    fig.text(0.67,0.93,'Interpolated')
    fig.text(0.82,0.93,'Reconstructed')
    if figname:
        plt.savefig('./figs/'+figname,bbox_inches='tight')
    else:
        plt.show()

In [ ]:
make_image_snapshots_clean(results_loss3, '2dkol_clean_snapshots'+str(plt_t), plt_t)

In [ ]:
spectrum_pred, kbins= get_tke(results_loss3[3], datainfo)
spectrum_ref, _ = get_tke(results_loss3[0], datainfo)
spectrum_interp, _ = get_tke(results_loss3[2], datainfo)

In [ ]:
probe = [15,39]

fig = plt.figure(figsize=(7,3))
ax1 = fig.add_subplot(122, anchor=(0.7,0.0))
# fig, (ax0, ax1) = plt.subplots(1,2,figsize=(7,3))
ax1.plot(kbins, spectrum_ref, label='Referece', color=grey, alpha=0.5, linewidth=3)
ax1.plot(kbins, spectrum_interp, label='Interpolated', color=my_discrete_cmap(0))
ax1.plot(kbins, spectrum_pred, label='Reconstructed', color=my_discrete_cmap(1),linestyle='--')
ax1.set(yscale='log', xscale='log', xlabel='wavenumber', ylabel='TKE')
ax1.set_xlim([1,k_nyquist])
ax1.legend()

ax01 = fig.add_subplot(321)
ax01.plot(results_loss3[0][:,probe[0],probe[0],0],color=grey,alpha=0.5,linewidth=5)
ax01.plot(results_loss3[2][:,probe[0],probe[0],0],color=my_discrete_cmap(0))
ax01.plot(results_loss3[3][:,probe[0],probe[0],0],color=my_discrete_cmap(1),linestyle='--')

ax02 = fig.add_subplot(323)
ax02.plot(results_loss3[0][:,probe[0],probe[0],1],color=grey,alpha=0.5,linewidth=5)
ax02.plot(results_loss3[2][:,probe[0],probe[0],1],color=my_discrete_cmap(0))
ax02.plot(results_loss3[3][:,probe[0],probe[0],1],color=my_discrete_cmap(1),linestyle='--')

ax03 = fig.add_subplot(325)
ax03.plot(results_loss3[0][:,probe[0],probe[0],2],color=grey,alpha=0.5,linewidth=5)
ax03.plot(results_loss3[2][:,probe[0],probe[0],2],color=my_discrete_cmap(0))
ax03.plot(results_loss3[3][:,probe[0],probe[0],2],color=my_discrete_cmap(1),linestyle='--')

fig.tight_layout()
plt.show()

## Plot edge case

In [ ]:
results_extra, datainfo_extra, observed_extra = get_summary_onecase('../local_results/2dkol/repeat_clean_minimum/extreme_case_testruns/k2rpb2pi3240628182435') #[clean,noisy,interp,pred]

In [ ]:
print(relative_error(results_extra[3],results_extra[0]))
print(relative_error(results_extra[2],results_extra[0]))

In [ ]:
# vorticity
vort_train_clean = vorticity(results_extra[0][...,:-1],datainfo_extra)
vort_train_pred = vorticity(results_extra[3][...,:-1],datainfo_extra)
vort_train_interp = vorticity(results_extra[2][...,:-1],datainfo_extra)
# spectrum
spectrum_pred_extra, kbins = get_tke(results_extra[3], datainfo_extra)
spectrum_ref, _ = get_tke(results_extra[0], datainfo_extra)
spectrum_interp_extra, _= get_tke(results_extra[2], datainfo_extra)

In [ ]:
plt_t_step = 1000

fig = plt.figure(figsize=(7,2.5))
grid = ImageGrid(fig,(0.05,0.1,0.45,0.85),(3,4),share_all=True,cbar_location='right',cbar_mode='single')
ax = fig.add_axes((0.68,0.2,0.3,0.7))

vmax = np.max(vort_train_clean[0:3*plt_t_step:plt_t_step,...])-1
vmin = np.min(vort_train_clean[0:3*plt_t_step:plt_t_step,...])+1
axes = grid.axes_all
for i in range(3):
    im0 = axes[i].imshow(vort_train_clean[plt_t_step*i,:,:].T,vmin=vmin,vmax=vmax)
    im1 = axes[i+4].imshow(vort_train_pred[plt_t_step*i,:,:].T,vmin=vmin,vmax=vmax)
    cbar = grid.cbar_axes[i].colorbar(im0)
    cbar = grid.cbar_axes[i].colorbar(im0,label='Vorticity')
    im2 = axes[i+8].imshow(vort_train_interp[plt_t_step*i,:,:].T,vmin=vmin,vmax=vmax)
    axes[i+8].set_xlabel(xlabel=f'$t={int(plt_t_step*i*datainfo_extra.dt)}$')
imm0 = axes[3].imshow(np.mean(vort_train_clean,axis=0).T,vmin=vmin,vmax=vmax)
imm1 = axes[7].imshow(np.mean(vort_train_pred,axis=0).T,vmin=vmin,vmax=vmax)
imm2 = axes[11].imshow(np.mean(vort_train_interp,axis=0).T,vmin=vmin,vmax=vmax)
axes[11].set_xlabel('Mean')


axes[0].set(yticks=[],ylabel='Ref.')
axes[0].spy(observed_extra[0,...,-1], color='r', marker='s', markersize=2, alpha=0.6, zorder=2)
axes[0].spy(observed_extra[0,...,0], color='k', marker='s', markersize=2, zorder=5)
axes[4].set(yticks=[],ylabel='Reconstructed')
axes[8].set(yticks=[],ylabel='Interp.')
for g in axes:
    g.set(xticks=[],xticklabels=[])
    g.tick_params(bottom=False,top=False)

ax.plot(kbins, spectrum_ref, label='Reference', color=grey, alpha=0.5, linewidth=3)
ax.plot(kbins, spectrum_interp_extra, label='Interpolated', color=my_discrete_cmap(0))
ax.plot(kbins, spectrum_pred_extra, label='Reconstructed', color=my_discrete_cmap(1),linestyle='--')
ax.set(yscale='log', xscale='log', xlabel='wavenumber', ylabel='TKE')
ax.set_xlim([1,k_nyquist])
ax.legend()

plt.savefig('./figs/2dkol_clean_10sensors_overall')
plt.show()

## Make title card

In [ ]:
example_pred = get_summary_onecase('../local_results/2dkol/repeat_clean_minimum/extreme_case_testruns/k2rpb2pi3240628182435',predict_only=True) #[clean,noisy,interp,pred]

In [ ]:
import itertools as it
example_sensors = [*it.permutations(range(128),2)]
np.random.shuffle(example_sensors)
example_sensors = np.array(example_sensors[:50])

In [ ]:
fig = plt.figure(figsize=(1.5,1.5))
ax = fig.add_subplot(111)
example_pred_masked = np.empty_like(example_pred[1000,:,:,-1])
ax.imshow(example_pred[1000,:,:,0],cmap='Greys',alpha=0.4)
ax.scatter(example_sensors[:,0],example_sensors[:,1],s=3,c=example_pred[1000,example_sensors[:,0],example_sensors[:,1],0],cmap='jet',marker='s')
ax.set(xticks=[],yticks=[])
fig.savefig('./figs/title_input',bbox_inches = 'tight', pad_inches = 0)
for i in range(3):
    fig = plt.figure(figsize=(1.5,1.5))
    ax = fig.add_subplot(111)
    im = ax.imshow(example_pred[1000,:,:,i],cmap='jet',origin='lower')
    ax.set(xticks=[],yticks=[])
    fig.savefig(f'./figs/title_output{i}', bbox_inches = 'tight', pad_inches = 0)


# Noisy cases

In [ ]:
def make_image_snapshots_vorticity(data, datainfo, figname, t1):
    # data is (ref, noisy, interp, classic, loss3, mean3)

    ref = data[0]

    fig = plt.figure(figsize=(7,7))
    
    # grids for mean
    grid_b1 = ImageGrid(fig, (0.08,0.00,0.92,0.22), (1,6),cbar_mode='single', share_all=True)
    grid_b2 = ImageGrid(fig, (0.08,0.23,0.92,0.22), (1,6),cbar_mode='single', share_all=True)
    
    # grids for snapshots
    grid_t1 = ImageGrid(fig, (0.08,0.52,0.92,0.22), (1,6),cbar_mode='single', share_all=True)
    grid_t2 = ImageGrid(fig, (0.08,0.75,0.92,0.22), (1,6),cbar_mode='single', share_all=True)
    
    data_vort = []
    for _data in data:
        data_vort.append(
            vorticity(_data[...,:2], datainfo)
        )
    
    axes = grid_t2.axes_all
    im_ref = axes[0].imshow(
        data_vort[0][t1,...].T
    )
    for j in range(1,6):
        im = axes[j].imshow(
            data_vort[j][t1,...].T
        )
        im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
    grid_t2.cbar_axes[0].colorbar(im_ref)
    grid_t2.axes_all[0].set(xticks=[],yticks=[])
    axes = grid_t1.axes_all
    im_ref = axes[0].imshow(
        ref[t1,...,2].T
    )
    for j in range(1,6):
        im = axes[j].imshow(
            data[j][t1,...,2].T
        )
        im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
    grid_t1.cbar_axes[0].colorbar(im_ref)
    grid_t1.axes_all[0].set(xticks=[],yticks=[])
    
    
    # mean
    axes = grid_b2.axes_all
    im_ref = axes[0].imshow(
        np.mean(data_vort[0],axis=0).T
    )
    for j in range(1,6):
        im = axes[j].imshow(
            np.mean(data_vort[j],axis=0).T
        )
        im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
    grid_b2.cbar_axes[0].colorbar(im_ref)
    grid_b2.axes_all[0].set(xticks=[],yticks=[])
    axes = grid_b1.axes_all
    im_ref = axes[0].imshow(
        np.mean(ref[...,2],axis=0).T
    )
    for j in range(1,6):
        im = axes[j].imshow(
            np.mean(data[j][...,2],axis=0).T
        )
        im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
    grid_b1.cbar_axes[0].colorbar(im_ref)
    grid_b1.axes_all[0].set(xticks=[],yticks=[])
    
 
    
    fig.text(0.11,0.98,'Reference')
    fig.text(0.28,0.98,'Noisy')
    fig.text(0.41,0.98,'Interpolated')
    fig.text(0.60,0.98,'$\mathcal{L}^c$')
    fig.text(0.75,0.98,'$\mathcal{L}^s$')
    fig.text(0.9,0.98,'$\mathcal{L}^m$')
    fig.text(0.005,0.20,'Mean',rotation='vertical')
    fig.text(0.05,0.1, '$\overline{p}$')
    fig.text(0.05,0.3, '$\overline{v}$')
    fig.text(0.005,0.70,f't={t1*datainfo.dt:.1f}',rotation='vertical')
    fig.text(0.05,0.6, '$p$')
    fig.text(0.05,0.85, '$v$')

    if figname:
        plt.savefig('./figs/'+figname,bbox_inches='tight')
    else:
        plt.show()

In [ ]:
def make_image_snapshots(data, datainfo, figname, t1):
    # data is (ref, noisy, interp, classic, loss3, mean3)

    ref = data[0]

    fig = plt.figure(figsize=(7,7))
    
    # grids for mean
    grid_b1 = ImageGrid(fig, (0.08,0.00,0.92,0.14), (1,6),cbar_mode='single', share_all=True)
    grid_b2 = ImageGrid(fig, (0.08,0.15,0.92,0.14), (1,6),cbar_mode='single', share_all=True)
    grid_b3 = ImageGrid(fig, (0.08,0.30,0.92,0.14), (1,6),cbar_mode='single', share_all=True)
    
    # grids for snapshots
    grid_t1 = ImageGrid(fig, (0.08,0.50,0.92,0.14), (1,6),cbar_mode='single', share_all=True)
    grid_t2 = ImageGrid(fig, (0.08,0.65,0.92,0.14), (1,6),cbar_mode='single', share_all=True)
    grid_t3 = ImageGrid(fig, (0.08,0.80,0.92,0.14), (1,6),cbar_mode='single', share_all=True)
    
    # snapshots
    for i, grid in enumerate([grid_t3,grid_t2,grid_t1]):
        axes = grid.axes_all
        im_ref = axes[0].imshow(ref[t1,...,i].T)
        
        for j in range(1,6):
            im = axes[j].imshow(data[j][t1,...,i].T)
            im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
        grid.cbar_axes[0].colorbar(im_ref)
        grid.axes_all[0].set(xticks=[],yticks=[])
    
    # mean
    for i, grid in enumerate([grid_b3,grid_b2,grid_b1]):
        axes = grid.axes_all
        im_ref = axes[0].imshow(np.mean(ref[...,i],axis=0).T)
        
        for j in range(1,6):
            im = axes[j].imshow(np.mean(data[j][...,i],axis=0).T)
            im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
        grid.cbar_axes[0].colorbar(im_ref)
        grid.axes_all[0].set(xticks=[],yticks=[])
    
    
    fig.text(0.11,0.98,'Reference')
    fig.text(0.28,0.98,'Noisy')
    fig.text(0.41,0.98,'Interpolated')
    fig.text(0.60,0.98,'$\mathcal{L}^c$')
    fig.text(0.75,0.98,'$\mathcal{L}^s$')
    fig.text(0.9,0.98,'$\mathcal{L}^m$')
    fig.text(0.01,0.20,'Mean',rotation='vertical')
    fig.text(0.05,0.07, '$\overline{p}$')
    fig.text(0.05,0.22, '$\overline{u}_2$')
    fig.text(0.05,0.37, '$\overline{u}_1$')
    fig.text(0.01,0.70,f't={t1*datainfo.dt:.1f}',rotation='vertical')
    fig.text(0.05,0.57, '$p$')
    fig.text(0.05,0.72, '$u_2$')
    fig.text(0.05,0.87, '$u_1$')

    if figname:
        plt.savefig('./figs/'+figname,bbox_inches='tight')
    else:
        plt.show()

In [ ]:
results_noisy_dir = Path('../local_results/2dkol/repeat_noisy')

psnr5 = re.compile('snr5')
psnr10 = re.compile('snr10')
psnr20 = re.compile('snr20')

def is_snr5(name):
    if re.search(psnr5, name):
        return True
    else:
        return False
def is_snr10(name):
    if re.search(psnr10, name):
        return True
    else:
        return False
def is_snr20(name):
    if re.search(psnr20, name):
        return True
    else:
        return False
p3n = re.compile('^loss3-')
def is_loss3_noisy(name):
    if re.search(p3n, name):
        return True
    else:
        return False

In [ ]:
def read_summary_noisy_all(d, lossfn):

    is_lossfn = {'loss3': is_loss3_noisy, 'lossmean': is_lossmean, 'lossclassic': is_lossclassic}[lossfn]

    with h5py.File(Path(d,'summary.h5')) as hf:
        names = np.array(hf.get('runs_name')).astype('unicode')
        l_train = np.array(hf.get('runs_loss_train'))

    lrel_20 = []
    lp_20 = []
    ltotal_20 = []
    name_20 = []
    lrel_10 = []
    lp_10 = []
    ltotal_10 = []
    name_10 = []
    lrel_5 = []
    lp_5 = []
    ltotal_5 = []
    name_5 = []

    for i in range(len(names)):
        _snr, _lossfn = names[i].split('_')
        if is_lossfn(_lossfn):
            if is_snr20(_snr):
                name_20.append(names[i])
                ltotal_20.append(np.sum(l_train[i,1:]))
                lrel_20.append(l_train[i,0])
                lp_20.append(np.sum(l_train[i,1:3]))
            if is_snr10(_snr):
                name_10.append(names[i])
                ltotal_10.append(np.sum(l_train[i,1:]))
                lrel_10.append(l_train[i,0])
                lp_10.append(np.sum(l_train[i,1:3]))
            if is_snr5(_snr):
                name_5.append(names[i])
                ltotal_5.append(np.sum(l_train[i,1:]))
                lrel_5.append(l_train[i,0])
                lp_5.append(np.sum(l_train[i,1:3]))
    ltotal = np.array([ltotal_20,ltotal_10,ltotal_5])
    lrel = np.array([lrel_20,lrel_10,lrel_5])
    lp = np.array([lp_20,lp_10,lp_5])
    name = np.array([name_20,name_10,name_5])
    
    mintotal = np.argmin(ltotal,axis=1)
    minrel = np.argmin(lrel,axis=1)

    print('Best run sorted by total loss is ', name[[0,1,2],mintotal])
    print('Best run sorted by relative loss is ', name[[0,1,2],minrel])

    return ltotal, lrel, lp, name

In [ ]:
def make_image_error_vorticity(data, datainfo, figname, t):
    ## results (ref, noisy, interp, classic, loss3, mean3)
    ref = data[0]
    data_vort = []
    for _data in data:
        data_vort.append(
            vorticity(_data[...,:2], datainfo).reshape(_data.shape[:-1]+(1,))
        )

    fig = plt.figure(figsize=(7.5,3.5))
    # grids for mean
    grid_b1 = ImageGrid(fig, (0.07,0.00,0.58,0.22), (1,6),cbar_mode='single', share_all=True)
    grid_b2 = ImageGrid(fig, (0.07,0.23,0.58,0.22), (1,6),cbar_mode='single', share_all=True)

    grid_b1e = ImageGrid(fig, (0.71,0.00,0.29,0.22), (1,3),cbar_mode='single', share_all=True)
    grid_b2e = ImageGrid(fig, (0.71,0.23,0.29,0.22), (1,3),cbar_mode='single', share_all=True)
    
    # grids for snapshots
    grid_t1 = ImageGrid(fig, (0.07,0.49,0.58,0.22), (1,6),cbar_mode='single', share_all=True)
    grid_t2 = ImageGrid(fig, (0.07,0.72,0.58,0.22), (1,6),cbar_mode='single', share_all=True)
    grid_t1e = ImageGrid(fig, (0.71,0.49,0.29,0.22), (1,3),cbar_mode='single', share_all=True)
    grid_t2e = ImageGrid(fig, (0.71,0.72,0.29,0.22), (1,3),cbar_mode='single', share_all=True)
    
    ## snapshots ### here -------------------
    # vorticity
    for _data, grid in zip([data_vort, data],[grid_t2, grid_t1]):
        axes = grid.axes_all 
        im_ref = axes[0].imshow(
            _data[0][t,...,-1].T
        )
        for j in range(1,6):
            im = axes[j].imshow(
                _data[j][t,...,-1].T
            )
            im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
        grid.cbar_axes[0].colorbar(im_ref)
        grid.axes_all[0].set(xticks=[],yticks=[])

    # # snapshot error
    for _data, grid in zip([data_vort, data],[grid_t2e,grid_t1e]):
        axes = grid.axes_all
        
        lvmin = []
        lvmax = []
        for j in range(3):
            im = axes[j].imshow(np.abs(_data[0][t,...,-1] - _data[j+3][t,...,-1]).T)
            lvmin.append(im.get_clim()[0])
            lvmax.append(im.get_clim()[1])
        for j in range(3):
            axes[j].get_images()[0].set_clim(np.min(lvmin), np.max(lvmax))
        grid.cbar_axes[0].colorbar(axes[0].get_images()[0])
        grid.axes_all[0].set(xticks=[],yticks=[])
    

    
    # # mean
    for _data, grid in zip([data_vort,data],[grid_b2,grid_b1]):
        axes = grid.axes_all
        im_ref = axes[0].imshow(np.mean(_data[0][...,-1],axis=0).T)
        
        for j in range(1,6):
            im = axes[j].imshow(np.mean(_data[j][...,-1],axis=0).T)
            im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
        grid.cbar_axes[0].colorbar(im_ref)
        grid.axes_all[0].set(xticks=[],yticks=[])
    # mean error
    for _data, grid in zip([data_vort,data],[grid_b2e,grid_b1e]):
        axes = grid.axes_all
        
        lvmin = []
        lvmax = []
        for j in range(3):
            im = axes[j].imshow(np.abs(np.mean(_data[0][...,-1],axis=0) - np.mean(_data[j+3][...,-1],axis=0)).T)
            lvmin.append(im.get_clim()[0])
            lvmax.append(im.get_clim()[1])
        for j in range(3):
            axes[j].get_images()[0].set_clim(np.min(lvmin), np.max(lvmax))
        grid.cbar_axes[0].colorbar(axes[0].get_images()[0])
        grid.axes_all[0].set(xticks=[],yticks=[])
    
    
    fig.text(0.09,0.98,'Reference')
    fig.text(0.20,0.98,'Noisy')
    fig.text(0.27,0.98,'Interpolated')
    fig.text(0.40,0.98,'$\mathcal{L}^c$')
    fig.text(0.49,0.98,'$\mathcal{L}^s$')
    fig.text(0.58,0.98,'$\mathcal{L}^m$')
    fig.text(0.01,0.20,'Mean',rotation='vertical')
    fig.text(0.04,0.1, '$\overline{p}$')
    fig.text(0.04,0.31, '$\overline{v}$')
    fig.text(0.01,0.70,f't={t*datainfo.dt:.1f}',rotation='vertical')
    fig.text(0.04,0.6, '$p$')
    fig.text(0.04,0.83, '$v$')
    fig.text(0.81,0.99, 'Absolute error')
    fig.text(0.75,0.95,'$\mathcal{L}^c$')
    fig.text(0.84,0.95,'$\mathcal{L}^s$')
    fig.text(0.93,0.95,'$\mathcal{L}^m$')

    if figname:
        plt.savefig('./figs/'+figname,bbox_inches='tight')
    else:
        plt.show()


In [ ]:
def make_image_error(data, datainfo, figname, t):
    ## results (ref, noisy, interp, classic, loss3, mean3)
    ref = data[0]

    fig = plt.figure(figsize=(7.5,5))
    # grids for mean
    grid_b1 = ImageGrid(fig, (0.07,0.00,0.58,0.14), (1,6),cbar_mode='single', share_all=True)
    grid_b2 = ImageGrid(fig, (0.07,0.15,0.58,0.14), (1,6),cbar_mode='single', share_all=True)
    grid_b3 = ImageGrid(fig, (0.07,0.30,0.58,0.14), (1,6),cbar_mode='single', share_all=True)

    grid_b1e = ImageGrid(fig, (0.71,0.00,0.29,0.14), (1,3),cbar_mode='single', share_all=True)
    grid_b2e = ImageGrid(fig, (0.71,0.15,0.29,0.14), (1,3),cbar_mode='single', share_all=True)
    grid_b3e = ImageGrid(fig, (0.71,0.30,0.29,0.14), (1,3),cbar_mode='single', share_all=True)
    
    # grids for snapshots
    grid_t1 = ImageGrid(fig, (0.07,0.50,0.58,0.14), (1,6),cbar_mode='single', share_all=True)
    grid_t2 = ImageGrid(fig, (0.07,0.65,0.58,0.14), (1,6),cbar_mode='single', share_all=True)
    grid_t3 = ImageGrid(fig, (0.07,0.80,0.58,0.14), (1,6),cbar_mode='single', share_all=True)
    grid_t1e = ImageGrid(fig, (0.71,0.50,0.29,0.14), (1,3),cbar_mode='single', share_all=True)
    grid_t2e = ImageGrid(fig, (0.71,0.65,0.29,0.14), (1,3),cbar_mode='single', share_all=True)
    grid_t3e = ImageGrid(fig, (0.71,0.80,0.29,0.14), (1,3),cbar_mode='single', share_all=True)
    
    # snapshots
    for i, grid in enumerate([grid_t3,grid_t2,grid_t1]):
        axes = grid.axes_all
        im_ref = axes[0].imshow(ref[t,...,i].T)
        
        for j in range(1,6):
            im = axes[j].imshow(data[j][t,...,i].T)
            im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
        grid.cbar_axes[0].colorbar(im_ref)
        grid.axes_all[0].set(xticks=[],yticks=[])
    # snapshot error
    for i, grid in enumerate([grid_t3e,grid_t2e,grid_t1e]):
        axes = grid.axes_all
        
        lvmin = []
        lvmax = []
        for j in range(3):
            im = axes[j].imshow(np.abs(ref[t,...,i] - data[j+3][t,...,i]).T)
            lvmin.append(im.get_clim()[0])
            lvmax.append(im.get_clim()[1])
        for j in range(3):
            axes[j].get_images()[0].set_clim(np.min(lvmin), np.max(lvmax))
        grid.cbar_axes[0].colorbar(axes[0].get_images()[0])
        grid.axes_all[0].set(xticks=[],yticks=[])
    

    
    # mean
    for i, grid in enumerate([grid_b3,grid_b2,grid_b1]):
        axes = grid.axes_all
        im_ref = axes[0].imshow(np.mean(ref[...,i],axis=0).T)
        
        for j in range(1,6):
            im = axes[j].imshow(np.mean(data[j][...,i],axis=0).T)
            im.set_clim(im_ref.get_clim()[0],im_ref.get_clim()[1])
        grid.cbar_axes[0].colorbar(im_ref)
        grid.axes_all[0].set(xticks=[],yticks=[])
    # mean error
    for i, grid in enumerate([grid_b3e,grid_b2e,grid_b1e]):
        axes = grid.axes_all
        
        lvmin = []
        lvmax = []
        for j in range(3):
            im = axes[j].imshow(np.abs(np.mean(ref[...,i],axis=0) - np.mean(data[j+3][...,i],axis=0)).T)
            lvmin.append(im.get_clim()[0])
            lvmax.append(im.get_clim()[1])
        for j in range(3):
            axes[j].get_images()[0].set_clim(np.min(lvmin), np.max(lvmax))
        grid.cbar_axes[0].colorbar(axes[0].get_images()[0])
        grid.axes_all[0].set(xticks=[],yticks=[])
    
    
    fig.text(0.09,0.98,'Reference')
    fig.text(0.20,0.98,'Noisy')
    fig.text(0.27,0.98,'Interpolated')
    fig.text(0.40,0.98,'$\mathcal{L}^c$')
    fig.text(0.49,0.98,'$\mathcal{L}^s$')
    fig.text(0.58,0.98,'$\mathcal{L}^m$')
    fig.text(0.01,0.20,'Mean',rotation='vertical')
    fig.text(0.04,0.07, '$\overline{p}$')
    fig.text(0.04,0.22, '$\overline{u}_2$')
    fig.text(0.04,0.37, '$\overline{u}_1$')
    fig.text(0.01,0.70,f't={t*datainfo.dt:.1f}',rotation='vertical')
    fig.text(0.04,0.57, '$p$')
    fig.text(0.04,0.72, '$u_2$')
    fig.text(0.04,0.87, '$u_1$')
    fig.text(0.81,0.99, 'Absolute error')
    fig.text(0.75,0.96,'$\mathcal{L}^c$')
    fig.text(0.84,0.96,'$\mathcal{L}^s$')
    fig.text(0.93,0.96,'$\mathcal{L}^m$')

    if figname:
        plt.savefig('./figs/'+figname,bbox_inches='tight')
    else:
        plt.show()


## Overall comparison all signal to noise ratio

In [ ]:
ltn_classic, lreln_classic, lpn_classic, namen_classic = read_summary_noisy_all(results_noisy_dir, 'lossclassic')
ltn_loss3, lreln_loss3, lpn_loss3, namen_loss3 = read_summary_noisy_all(results_noisy_dir, 'loss3')
ltn_mean, lreln_mean, lpn_mean, namen_mean = read_summary_noisy_all(results_noisy_dir, 'lossmean')

In [ ]:
seeds = '7521-42135'
best_noisy_run = [
    ['snr20_classic-'+seeds, 'snr10_classic-'+seeds, 'snr5_classic-'+seeds],
    ['snr20_loss3-'+seeds, 'snr10_loss3-'+seeds, 'snr5_loss3-'+seeds],
    ['snr20_mean3-'+seeds, 'snr10_mean3-'+seeds, 'snr5_mean3-'+seeds]
] #[lossfn, snr]

In [ ]:
snr = [20,10,5]
lp_ref = 1.1317265e-05

fig, (axl, axr) = plt.subplots(1,2,figsize=(7,2.5), sharex=True)

mean_lreln_classic = np.mean(lreln_classic*100,axis=1)
std_lreln_classic = np.std(lreln_classic*100,axis=1)
mean_lreln_loss3 = np.mean(lreln_loss3*100,axis=1)
std_lreln_loss3 = np.std(lreln_loss3*100,axis=1)
mean_lreln_mean = np.mean(lreln_mean*100,axis=1)
std_lreln_mean = np.std(lreln_mean*100,axis=1)

mean_lpn_classic = np.mean(lpn_classic,axis=1)
std_lpn_classic = np.std(lpn_classic,axis=1)
mean_lpn_loss3 = np.mean(lpn_loss3,axis=1)
std_lpn_loss3 = np.std(lpn_loss3,axis=1)
mean_lpn_mean = np.mean(lpn_mean,axis=1)
std_lpn_mean = np.std(lpn_mean,axis=1)

axl.errorbar(snr,mean_lreln_classic,yerr=std_lreln_classic,label='$\mathcal{L}^c$ ',marker='x',color=my_discrete_cmap(0),linewidth=2.5)
axl.errorbar(snr,mean_lreln_loss3,yerr=std_lreln_loss3,label='$\mathcal{L}^s$ ',marker='x',color=my_discrete_cmap(1),linewidth=2.5)
axl.errorbar(snr,mean_lreln_mean,yerr=std_lreln_mean,label='$\mathcal{L}^m$ ',marker='x',color=my_discrete_cmap(2),linewidth=2.5)

axl.set_ylabel('$\epsilon (\%)$')
axl.set_xticks([5,10,20])
axl.set_xlabel('SNR')

axr.errorbar(snr,np.mean(lpn_classic,axis=1),yerr=np.std(lpn_classic,axis=1),marker='x',color=my_discrete_cmap(0),linewidth=2.5)
axr.errorbar(snr,np.mean(lpn_loss3,axis=1),yerr=np.std(lpn_loss3,axis=1), marker='x',color=my_discrete_cmap(1),linewidth=2.5)
axr.errorbar(snr,np.mean(lpn_mean,axis=1),yerr=np.std(lpn_mean,axis=1),marker='x',color=my_discrete_cmap(2),linewidth=2.5)
axr.hlines(lp_ref, xmin=5,xmax=20, colors=['k'], linestyles='dashed',label='reference data')
# axr.legend(ncol=1)
axr.set_ylabel('$\mathcal{L}_p$')
axr.set_xticks([5,10,20])
axr.set_xlabel('SNR')

fig.legend(ncol=4,loc='upper center', bbox_to_anchor=(0.5, 1.1))
fig.tight_layout()
fig.savefig('./figs/2dkol_noisy_compare_lossfn',bbox_inches='tight')
plt.show()

print(f'Relative error')
print(f'        SNR          20            10               5')
print(f'lossclassic  {mean_lreln_classic[0]:.3f}+-{std_lreln_classic[0]:.5f}  {mean_lreln_classic[1]:.3f}+-{std_lreln_classic[1]:.5f}  {mean_lreln_classic[2]:.3f}+-{std_lreln_classic[2]:.5f}')
print(f'loss3        {mean_lreln_loss3[0]:.3f}+-{std_lreln_loss3[0]:.5f}  {mean_lreln_loss3[1]:.3f}+-{std_lreln_loss3[1]:.5f}  {mean_lreln_loss3[2]:.3f}+-{std_lreln_loss3[2]:.5f}')
print(f'lossmean     {mean_lreln_mean[0]:.3f}+-{std_lreln_mean[0]:.5f}  {mean_lreln_mean[1]:.3f}+-{std_lreln_mean[1]:.5f}  {mean_lreln_mean[2]:.3f}+-{std_lreln_mean[2]:.5f}')

print(f'Physics loss')
print(f'        SNR          20            10               5')
print(f'lossclassic  {mean_lpn_classic[0]:.3f}+-{std_lpn_classic[0]:.5f}  {mean_lpn_classic[1]:.3f}+-{std_lpn_classic[1]:.5f}  {mean_lpn_classic[2]:.3f}+-{std_lpn_classic[2]:.5f}')
print(f'loss3        {mean_lpn_loss3[0]:.3f}+-{std_lpn_loss3[0]:.5f}  {mean_lpn_loss3[1]:.3f}+-{std_lpn_loss3[1]:.5f}  {mean_lpn_loss3[2]:.3f}+-{std_lpn_loss3[2]:.5f}')
print(f'lossmean     {mean_lpn_mean[0]:.3f}+-{std_lpn_mean[0]:.5f}  {mean_lpn_mean[1]:.3f}+-{std_lpn_mean[1]:.5f}  {mean_lpn_mean[2]:.3f}+-{std_lpn_mean[2]:.5f}')

## Plots of cases

In [ ]:
plt_tn = 1931

### SNR10

In [ ]:
results10, datainfo, observed = get_summary_onecase(Path(results_noisy_dir,best_noisy_run[0][1]))
_results10_loss3 = get_summary_onecase(Path(results_noisy_dir,best_noisy_run[1][1]), predict_only=True)
_results10_mean3 = get_summary_onecase(Path(results_noisy_dir,best_noisy_run[2][1]), predict_only=True)
results10 = list(results10)
results10.append(_results10_loss3)
results10.append(_results10_mean3)

In [ ]:
make_image_sensor_locations(results10, observed, plt_tn, '2dkol_noisy_sensor_location')

In [ ]:
# make_image_error(results10,datainfo,'2dkol_noisy10_snapshots'+str(plt_tn),plt_tn)
make_image_error_vorticity(results10,datainfo,'2dkol_noisy10_snapshots_vorticity'+str(plt_tn),plt_tn)

### SNR 5

In [ ]:
results5, datainfo, observed = get_summary_onecase(Path(results_noisy_dir,best_noisy_run[0][2]))
_results5_loss3 = get_summary_onecase(Path(results_noisy_dir,best_noisy_run[1][2]), predict_only=True)
_results5_mean3 = get_summary_onecase(Path(results_noisy_dir,best_noisy_run[2][2]), predict_only=True)
results5 = list(results5)
results5.append(_results5_loss3)
results5.append(_results5_mean3)

In [ ]:
# make_image_error(results5,datainfo,'2dkol_noisy5_snapshots'+str(plt_tn),plt_tn)
make_image_error_vorticity(results5,datainfo,'2dkol_noisy5_snapshots_vorticity'+str(plt_tn),plt_tn)

In [ ]:
relative_error(results5[-1],results5[0])

### SNR20

In [ ]:
results20, datainfo, observed = get_summary_onecase(Path(results_noisy_dir,best_noisy_run[0][0]))
_results20_loss3 = get_summary_onecase(Path(results_noisy_dir,best_noisy_run[1][0]), predict_only=True)
_results20_mean3 = get_summary_onecase(Path(results_noisy_dir,best_noisy_run[2][0]), predict_only=True)
results20 = list(results20)
results20.append(_results20_loss3)
results20.append(_results20_mean3)

In [ ]:
# make_image_error(results20,datainfo,'2dkol_noisy20_snapshots'+str(plt_tn),plt_tn)
make_image_error_vorticity(results20,datainfo,'2dkol_noisy20_snapshots_vorticity'+str(plt_tn),plt_tn)

all snr

In [ ]:
## results (ref, noisy, interp, classic, loss3, mean3)

spectrum_ref, kbins = get_tke(results20[0]-np.mean(results20[0],axis=0,keepdims=True), datainfo)
spectrum_interp20, _ = get_tke(results20[2]-np.mean(results20[2],axis=0,keepdims=True), datainfo)
spectrum_lossclassic20, _ = get_tke(results20[3]-np.mean(results20[3],axis=0,keepdims=True), datainfo)
spectrum_loss320, _ = get_tke(results20[4]-np.mean(results20[4],axis=0,keepdims=True), datainfo)
spectrum_lossmean20, _ = get_tke(results20[5]-np.mean(results20[5],axis=0,keepdims=True), datainfo)

spectrum_interp10, _ = get_tke(results10[2]-np.mean(results10[2],axis=0,keepdims=True), datainfo)
spectrum_lossclassic10, _ = get_tke(results10[3]-np.mean(results10[3],axis=0,keepdims=True), datainfo)
spectrum_loss310, _ = get_tke(results10[4]-np.mean(results10[4],axis=0,keepdims=True), datainfo)
spectrum_lossmean10, _ = get_tke(results10[5]-np.mean(results10[5],axis=0,keepdims=True), datainfo)

spectrum_interp5, _ = get_tke(results5[2]-np.mean(results5[2],axis=0,keepdims=True), datainfo)
spectrum_lossclassic5, _ = get_tke(results5[3]-np.mean(results5[3],axis=0,keepdims=True), datainfo)
spectrum_loss35, _ = get_tke(results5[4]-np.mean(results5[4],axis=0,keepdims=True), datainfo)
spectrum_lossmean5, _ = get_tke(results5[5]-np.mean(results5[5],axis=0,keepdims=True), datainfo)

In [ ]:
fig1,axes = plt.subplots(1,3,figsize=(7,3),sharey=True)

# snr20

line0, = axes[0].loglog(kbins, spectrum_ref, label='Referece', color=grey, alpha=0.5, linewidth=3)
line1, = axes[0].loglog(kbins, spectrum_interp20, label='Interpolated', color='k',linestyle='--')
line2, = axes[0].loglog(kbins, spectrum_lossclassic20, label='$\mathcal{L}^c$', color=my_discrete_cmap(0))
line3, = axes[0].loglog(kbins, spectrum_loss320, label='$\mathcal{L}^s$', color=my_discrete_cmap(1))
line4, = axes[0].loglog(kbins, spectrum_lossmean20, label='$\mathcal{L}^m$', color=my_discrete_cmap(2))
axes[0].set(xlabel='wavenumber', ylabel='TKE')
axes[0].set_xlim([1,k_nyquist])
axes[0].set_title('SNR20')
axes[0].grid(axis='x',which='minor')

axes[1].loglog(kbins, spectrum_ref, color=grey, alpha=0.5, linewidth=3)
axes[1].loglog(kbins, spectrum_interp10, color='k',linestyle='--')
axes[1].loglog(kbins, spectrum_lossclassic10, color=my_discrete_cmap(0))
axes[1].loglog(kbins, spectrum_loss310, color=my_discrete_cmap(1))
axes[1].loglog(kbins, spectrum_lossmean10, color=my_discrete_cmap(2))
axes[1].set(xlabel='wavenumber')
axes[1].set_xlim([1,k_nyquist])
axes[1].set_title('SNR10')
axes[1].grid(axis='x',which='minor')


axes[2].loglog(kbins, spectrum_ref, color=grey, alpha=0.5, linewidth=3)
axes[2].loglog(kbins, spectrum_interp5, color='k',linestyle='--')
axes[2].loglog(kbins, spectrum_lossclassic5, color=my_discrete_cmap(0))
axes[2].loglog(kbins, spectrum_loss35, color=my_discrete_cmap(1))
axes[2].loglog(kbins, spectrum_lossmean5, color=my_discrete_cmap(2))
axes[2].set(xlabel='wavenumber')
axes[2].set_xlim([1,k_nyquist])
axes[2].set_title('SNR5')
axes[2].grid(axis='x',which='minor')

fig1.legend(handles = [line0, line1, line2, line3, line4], ncol=5,loc='upper center', bbox_to_anchor=(0.5, 1.11))
fig1.savefig('./figs/2dkol_noisy_tke',bbox_inches='tight')
plt.show()

In [ ]:
probex = 45
probey = 94

fig,axes = plt.subplots(3,1, figsize=(7,3.5), sharey=True, sharex=True)
realt = np.arange(results20[0].shape[0])*datainfo.dt
snr = [20,10,5]

for i, _data in enumerate([results20,results10,results5]):
    axes[i].plot(realt,_data[1][:,probex,probey,0], label='Noisy',color=grey,alpha=0.5)
    axes[i].plot(realt,_data[2][:,probex,probey,0], label='Interpolated', color='k',alpha=0.5)
    axes[i].plot(realt,_data[4][:,probex,probey,0], label='$\mathcal{L}^s$', color=my_discrete_cmap(1), linewidth=1, linestyle='--')
    axes[i].plot(realt,_data[0][:,probex,probey,0], label='Referece', color='r')
    axes[i].plot(realt,_data[5][:,probex,probey,0], label='$\mathcal{L}^m$', color=my_discrete_cmap(2), linewidth=1.5)
    axes[i].plot(realt,_data[3][:,probex,probey,0], label='$\mathcal{L}^c$', color=my_discrete_cmap(0), linewidth=1.5, linestyle='-.')
    axes[i].set_title(f'SNR {snr[i]}')
    axes[i].set_ylabel('$u_1$')

axes[2].set_xlabel('time (s)')
axes[2].set_xlim([0,100])
handles,labels = axes[0].get_legend_handles_labels()
fig.legend(handles=handles, ncol=6,loc='upper center', bbox_to_anchor=(0.5, 1.1))
plt.show()

## Very sparse sensors

In [ ]:
dir_very_sparse = Path(results_noisy_dir,'min_observations_snr10')
very_sparse_runs = ['loss1_24inn_24sensors-7521-42135_1','loss3_24inn_24sensors-7521-42135_1/restart','mean3_24inn_24sensors-7521-42135_1']

In [ ]:
results_sparse, datainfo, observed_sparse = get_summary_onecase(Path(dir_very_sparse,very_sparse_runs[0]))
_results_sparse_loss3 = get_summary_onecase(Path(dir_very_sparse,very_sparse_runs[1]), predict_only=True)
_results_sparse_mean3 = get_summary_onecase(Path(dir_very_sparse,very_sparse_runs[2]), predict_only=True)
results_sparse = list(results_sparse)
results_sparse.append(_results_sparse_loss3)
results_sparse.append(_results_sparse_mean3)
# results_sparse is (ref, noisy, interp, classic, loss3, mean3)
for i in range(2,6):
    print(relative_error(results_sparse[i],results_sparse[0])*100)

In [ ]:
make_image_sensor_locations(results_sparse, observed_sparse, plt_tn, None)

In [ ]:
f = kolsol_forcing_term(4, 128, 2)
print(momentum_loss(results_sparse[-3], datainfo, forcing=f) + divergence(results_sparse[-3][...,:-1], datainfo))
print(momentum_loss(results_sparse[-2], datainfo, forcing=f) + divergence(results_sparse[-2][...,:-1], datainfo))
print(momentum_loss(results_sparse[-1], datainfo, forcing=f) + divergence(results_sparse[-1][...,:-1], datainfo))

In [ ]:
make_image_error(results_sparse,datainfo,None,plt_tn)
make_image_error_vorticity(results_sparse,datainfo,None,plt_tn)

In [ ]:
spectrum_ref, kbins = get_tke(results_sparse[0]-np.mean(results_sparse[0],axis=0,keepdims=True), datainfo)
spectrum_sparse_interp, _ = get_tke(results_sparse[2]-np.mean(results_sparse[2],axis=0,keepdims=True), datainfo)
spectrum_sparse_classic, _ = get_tke(results_sparse[3]-np.mean(results_sparse[3],axis=0,keepdims=True), datainfo)
spectrum_sparse_loss3, _ = get_tke(results_sparse[4]-np.mean(results_sparse[4],axis=0,keepdims=True), datainfo)
spectrum_sparse_lossmean, _ = get_tke(results_sparse[5]-np.mean(results_sparse[5],axis=0,keepdims=True), datainfo)
k_nyquist = (2*np.pi / np.sqrt(2*(datainfo.dx**2))) / 2.

In [ ]:
fig = plt.figure(figsize=(7.5,3)) 

grid1 = ImageGrid(fig,(0.07,0.05,0.60,0.4),(1,5),share_all=True,cbar_location='right',cbar_mode='single')
grid2 = ImageGrid(fig,(0.07,0.5,0.60,0.35),(1,5),share_all=True,cbar_location='right',cbar_mode='single')
ax1 = fig.add_axes((0.78,0.2,0.21,0.5))

im_ref1 = grid1.axes_all[0].imshow(np.mean(results_sparse[0][...,0],axis=0).T)
im_ref2 = grid2.axes_all[0].imshow(results_sparse[0][plt_tn,...,0].T)
grid1.axes_all[0].spy(observed_sparse[0,...,-1], color='r', marker='s', markersize=1, alpha=0.6, zorder=2)
grid1.axes_all[0].spy(observed_sparse[0,...,0], color='k', marker='s', markersize=1, zorder=5)
grid2.axes_all[0].spy(observed_sparse[0,...,-1], color='r', marker='s', markersize=1, alpha=0.6, zorder=2)
grid2.axes_all[0].spy(observed_sparse[0,...,0], color='k', marker='s', markersize=1, zorder=5)
vmin1,vmax1 = im_ref1.get_clim()
vmin2,vmax2 = im_ref2.get_clim()
for _i, (g1,g2) in enumerate(zip(grid1.axes_all[1:], grid2.axes_all[1:])):
    i = _i+2
    g1.imshow(np.mean(results_sparse[i][...,0],axis=0).T,vmin=vmin1,vmax=vmax1)
    g2.imshow(results_sparse[i][plt_tn,...,0].T,vmin=vmin2,vmax=vmax2)
grid1.cbar_axes[0].colorbar(im_ref1)
grid1.axes_all[0].set(xticks=[],yticks=[])
grid2.cbar_axes[0].colorbar(im_ref2)
grid2.axes_all[0].set(xticks=[],yticks=[])


l1,  = ax1.loglog(kbins, spectrum_ref, color=grey, alpha=0.5, linewidth=3, label='Ref.')
l2, = ax1.loglog(kbins, spectrum_sparse_interp, color='k',linestyle='--', label='Interp.')
l3, = ax1.loglog(kbins, spectrum_sparse_classic, color=my_discrete_cmap(0), label='$\mathcal{L}^c$')
l4, = ax1.loglog(kbins, spectrum_sparse_loss3, color=my_discrete_cmap(1), label='$\mathcal{L}^s$')
l5, = ax1.loglog(kbins, spectrum_sparse_lossmean, color=my_discrete_cmap(2), label='$\mathcal{L}^m$')
ax1.set_xlim([1,k_nyquist])
ax1.set_yticks([10e-6,10e7])
ax1.set_ylabel('TKE', labelpad=-9)
ax1.set_xlabel('wavenumber')
ax1.grid(axis='x',which='minor')
ax1.legend(handles=[l3,l4,l5,l1,l2],ncol=2,loc='upper center', bbox_to_anchor=(0.45, 1.5),fontsize='small')


fig.text(0.09,0.9,'Reference')
fig.text(0.20,0.9,'Interpolated')
fig.text(0.36,0.9,'$\mathcal{L}^c$')
fig.text(0.48,0.9,'$\mathcal{L}^s$')
fig.text(0.60,0.9,'$\mathcal{L}^m$')
fig.text(0.01,0.20,'Mean',rotation='vertical')
fig.text(0.04,0.25, '$\overline{u}_1$')
fig.text(0.01,0.6,f't={plt_tn*datainfo.dt:.1f}',rotation='vertical')
fig.text(0.04,0.68, '$u_1$')

fig.savefig('./figs/2dkol_noisy_sparse_overall')

In [ ]:
## ZOOM IN ON THE LOW WAVENUMBER
# fig,ax1 = plt.subplots(1,1,figsize=(4,3))
# l1,  = ax1.loglog(kbins, spectrum_ref, color=grey, alpha=0.5, linewidth=3, label='Ref.')
# l2, = ax1.loglog(kbins, spectrum_sparse_interp, color='k',linestyle='--', label='Interp.')
# l3, = ax1.loglog(kbins, spectrum_sparse_classic, color=my_discrete_cmap(0), label='$\mathcal{L}^c$')
# l4, = ax1.loglog(kbins, spectrum_sparse_loss3, color=my_discrete_cmap(1), label='$\mathcal{L}^s$')
# l5, = ax1.loglog(kbins, spectrum_sparse_lossmean, color=my_discrete_cmap(2), label='$\mathcal{L}^m$')
# ax1.set_xlim([1,15])
# ax1.set_ylim([11, 10e8])
# ax1.set_yticks([11,10e7])
# ax1.set_ylabel('TKE', labelpad=-9)
# ax1.set_xlabel('wavenumber')
# ax1.grid(axis='x',which='minor')
# ax1.legend(handles=[l3,l4,l5,l1,l2],ncol=2,loc='upper center', bbox_to_anchor=(0.45, 1.5),fontsize='small')


In [ ]:
fig, axes = plt.subplots(1,2, sharex=True, figsize=(7.5,2.3))
_labels = ['$\mathcal{L}^c$', '$\mathcal{L}^s$', '$\mathcal{L}^m$']


for i in [0,1,2]:
    if i == 1:
        with h5py.File(Path(dir_very_sparse,very_sparse_runs[i],'../results.h5')) as hf:
            loss_train = np.array(hf.get("loss_train"))
            loss_div = np.array(hf.get("loss_div"))
            loss_momentum = np.array(hf.get("loss_momentum"))
            loss_sensors = np.array(hf.get("loss_sensors"))
        with h5py.File(Path(dir_very_sparse,very_sparse_runs[i],'results.h5')) as hf:
            loss_train = np.concatenate((loss_train,np.array(hf.get("loss_train"))))
            loss_div = np.concatenate((loss_div,np.array(hf.get("loss_div"))))
            loss_momentum = np.concatenate((loss_momentum,np.array(hf.get("loss_momentum"))))
            loss_sensors = np.concatenate((loss_sensors,np.array(hf.get("loss_sensors"))))
        epochs = np.arange(0,len(loss_train),50)
    else:
        with h5py.File(Path(dir_very_sparse,very_sparse_runs[i],'results.h5')) as hf:
            loss_train = np.array(hf.get("loss_train"))
            loss_div = np.array(hf.get("loss_div"))
            loss_momentum = np.array(hf.get("loss_momentum"))
            loss_sensors = np.array(hf.get("loss_sensors"))
        epochs = np.arange(0,len(loss_train),50)
    axes[0].semilogy(epochs, loss_train[::50], color=my_discrete_cmap(i), label=_labels[i])
    axes[1].semilogy(epochs, (loss_div+loss_momentum)[::50], color=my_discrete_cmap(i))
    axes[1].semilogy(epochs, loss_sensors[::50], color=my_discrete_cmap(i), linestyle=':')
    # axes[2].semilogy(epochs, (loss_div+loss_momentum+loss_sensors)[::50], color=my_discrete_cmap(i))
    print('physics loss at epoch 5000 and 18000: ',loss_momentum[[5000, 18000]]+loss_div[[5000, 18000]],'data loss at epoch 5000 and 18000: ', loss_sensors[[5000, 18000]])
fig.legend(ncols=3, loc='upper center', bbox_to_anchor=(0.5,1.03))
axes[0].set(xlim=[0,30000], xlabel='Epochs', ylabel='Loss')
axes[1].set(xlim=[0,30000], xlabel='Epochs', ylabel='$\mathcal{L}_p$ (---), $\mathcal{L}_o$ ($\cdot \! \, \cdot \! \, \cdot$)') #'Physics (-) /\n sensor (..) loss'
# axes[2].set(xlabel='Epochs', ylabel='$\mathcal{L}_p + \mathcal{L}_o$') # 'Total unweighted loss'
fig.subplots_adjust(left=0.08,wspace=0.3,right=0.97,bottom=0.2,top=0.85)

fig.savefig('figs/2dkol_noisy_sparse_learning_curve')
# fig.savefig('learning_curve.png',bbox_inches='tight')